<a href="https://colab.research.google.com/github/Kzis/mrt-is/blob/master/Prep_Model_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# อ่านข้อมูล Mrt Transaction

In [30]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [32]:
# df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv" , nrows=10000)
df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv")

In [33]:
df.head()

,TransactionID,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
0,40,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
1,69,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
2,127,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
3,129,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
4,333,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017


# เลือกข้อมูลตามเดือนที่ต้องการ

In [34]:
def get_data_by_month(month):
  return df.loc[df['ThaiMonthName'] == month]

In [35]:
jan_df = get_data_by_month("มกราคม")
jan_df.drop('TransactionID', axis=1, inplace=True)
jan_df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
0,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
1,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
2,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
3,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
4,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017


# Count ข้อมูลแต่ละสถานี แยกตามแต่ละวัน

In [36]:
tran_df = jan_df[['TransactionDate','StationName','CardType']].groupby(['TransactionDate','StationName']).count()
tran_df.reset_index(inplace=True)
tran_df = tran_df.rename(columns={'CardType': 'PassengerCount'})
tran_df

,TransactionDate,StationName,PassengerCount
0,2017-01-01,Bang Sue,186
1,2017-01-01,Chatuchak Park,359
2,2017-01-01,Hua Lamphong,299
3,2017-01-01,Huai Khwang,431
4,2017-01-01,Kamphaeng Phet,233
...,...,...,...
553,2017-01-31,Sam Yan,1241
554,2017-01-31,Si Lom,2384
555,2017-01-31,Sukhumvit,4891
556,2017-01-31,Sutthisan,1968


In [37]:

def get_transaction_df(month_df):
  df_month_transaction_date_sum = month_df.TransactionDate.value_counts().rename_axis('TransactionDate').reset_index(name='TransactionCounts')
  df_month_transaction_date_sum.sort_values('TransactionDate')
  return df_month_transaction_date_sum

In [38]:
transaction_df = get_transaction_df(jan_df)
transaction_df.head()

,TransactionDate,TransactionCounts
0,2017-01-25,37135
1,2017-01-26,36485
2,2017-01-13,36171
3,2017-01-24,36050
4,2017-01-20,35898


# Preparation ข้อมูลประเภทของวัน (วันทำงาน , วันหยุด , วันนักขัตฤกษ์)

In [39]:
def get_day_type_list(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    print(col_list)
    output = []
    for date in date_list:
      lst = []
      lst.append(date)
      for col_val in col_list:
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.append(column_count.index[0])
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
      # print("-----")
      output.append(lst)
    print(output)
    return output
    # df = pd.DataFrame(output, columns =['TransactionDate','Type', 'Station','Count']) 
    # return df

In [40]:
day_type_list = get_day_type_list(jan_df , transaction_df , "DayType")

['WE', 'PH', 'WD']
[['2017-01-25', 0, 0, 37135], ['2017-01-26', 0, 0, 36485], ['2017-01-13', 0, 0, 36171], ['2017-01-24', 0, 0, 36050], ['2017-01-20', 0, 0, 35898], ['2017-01-11', 0, 0, 35831], ['2017-01-18', 0, 0, 35575], ['2017-01-10', 0, 0, 35343], ['2017-01-31', 0, 0, 35253], ['2017-01-19', 0, 0, 35208], ['2017-01-17', 0, 0, 35107], ['2017-01-09', 0, 0, 34666], ['2017-01-23', 0, 0, 34660], ['2017-01-06', 0, 0, 33946], ['2017-01-27', 0, 0, 33717], ['2017-01-16', 0, 0, 33200], ['2017-01-05', 0, 0, 33129], ['2017-01-30', 0, 0, 32814], ['2017-01-12', 0, 0, 31840], ['2017-01-04', 0, 0, 31479], ['2017-01-28', 16397, 0, 0], ['2017-01-21', 16184, 0, 0], ['2017-01-14', 16018, 0, 0], ['2017-01-07', 15156, 0, 0], ['2017-01-29', 13026, 0, 0], ['2017-01-03', 0, 12134, 0], ['2017-01-22', 12071, 0, 0], ['2017-01-15', 12068, 0, 0], ['2017-01-08', 11123, 0, 0], ['2017-01-02', 0, 7217, 0], ['2017-01-01', 5045, 0, 0]]


In [41]:
day_type_temp = pd.DataFrame(day_type_list, columns =['Date','WE','PH','WD'])
day_type_temp['DayType'] = ( (day_type_temp["WE"] != 0) * 1 ) + ( (day_type_temp["PH"] != 0) * 2 ) + ( (day_type_temp["WD"] != 0) * 3 )
day_type_temp.loc[day_type_temp['DayType'] == 1, 'DayType'] = "WE"
day_type_temp.loc[day_type_temp['DayType'] == 2, 'DayType'] = "PH"
day_type_temp.loc[day_type_temp['DayType'] == 3, 'DayType'] = "WD"
day_type_temp.drop('WE', axis=1, inplace=True)
day_type_temp.drop('PH', axis=1, inplace=True)
day_type_temp.drop('WD', axis=1, inplace=True)

In [42]:
day_type_temp.head()

,Date,DayType
0,2017-01-25,WD
1,2017-01-26,WD
2,2017-01-13,WD
3,2017-01-24,WD
4,2017-01-20,WD


In [43]:
jan_transaction_df = pd.merge(left=tran_df, right=day_type_temp, left_on='TransactionDate', right_on='Date')
jan_transaction_df.drop('Date', axis=1, inplace=True)
jan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType
0,2017-01-01,Bang Sue,186,WE
1,2017-01-01,Chatuchak Park,359,WE
2,2017-01-01,Hua Lamphong,299,WE
3,2017-01-01,Huai Khwang,431,WE
4,2017-01-01,Kamphaeng Phet,233,WE
...,...,...,...,...
553,2017-01-31,Sam Yan,1241,WD
554,2017-01-31,Si Lom,2384,WD
555,2017-01-31,Sukhumvit,4891,WD
556,2017-01-31,Sutthisan,1968,WD


# Preparation ข้อมูลวันอะไรของสัปดาห์

In [44]:
def get_day_of_week_list(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    print(col_list)
    output = []
    for date in date_list:
      lst = []
      lst.append(date)
      for col_val in col_list:
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.append(column_count.index[0])
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
      # print("-----")
      output.append(lst)
    print(output)
    return output

In [45]:
day_of_week_list = get_day_of_week_list(jan_df , transaction_df , "EnglishDayNameOfWeek")

['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
[['2017-01-25', 0, 0, 0, 37135, 0, 0, 0], ['2017-01-26', 0, 0, 0, 0, 36485, 0, 0], ['2017-01-13', 0, 0, 0, 0, 0, 36171, 0], ['2017-01-24', 0, 0, 36050, 0, 0, 0, 0], ['2017-01-20', 0, 0, 0, 0, 0, 35898, 0], ['2017-01-11', 0, 0, 0, 35831, 0, 0, 0], ['2017-01-18', 0, 0, 0, 35575, 0, 0, 0], ['2017-01-10', 0, 0, 35343, 0, 0, 0, 0], ['2017-01-31', 0, 0, 35253, 0, 0, 0, 0], ['2017-01-19', 0, 0, 0, 0, 35208, 0, 0], ['2017-01-17', 0, 0, 35107, 0, 0, 0, 0], ['2017-01-09', 0, 34666, 0, 0, 0, 0, 0], ['2017-01-23', 0, 34660, 0, 0, 0, 0, 0], ['2017-01-06', 0, 0, 0, 0, 0, 33946, 0], ['2017-01-27', 0, 0, 0, 0, 0, 33717, 0], ['2017-01-16', 0, 33200, 0, 0, 0, 0, 0], ['2017-01-05', 0, 0, 0, 0, 33129, 0, 0], ['2017-01-30', 0, 32814, 0, 0, 0, 0, 0], ['2017-01-12', 0, 0, 0, 0, 31840, 0, 0], ['2017-01-04', 0, 0, 0, 31479, 0, 0, 0], ['2017-01-28', 0, 0, 0, 0, 0, 0, 16397], ['2017-01-21', 0, 0, 0, 0, 0, 0, 16184], ['2017-01-14', 0, 

In [46]:
day_of_week_unique = jan_df['EnglishDayNameOfWeek'].unique().tolist()
day_of_week_unique.insert(0,"Date")

In [47]:
day_of_week_temp = pd.DataFrame(day_of_week_list, columns =day_of_week_unique)
day_of_week_temp['DateOfWeek'] = ( (day_of_week_temp["Sunday"] != 0) * 1 ) + ( (day_of_week_temp["Monday"] != 0) * 2 ) + ( (day_of_week_temp["Tuesday"] != 0) * 3 )+ ( (day_of_week_temp["Wednesday"] != 0) * 4 )+ ( (day_of_week_temp["Thursday"] != 0) * 5 )+ ( (day_of_week_temp["Friday"] != 0) * 6 )+ ( (day_of_week_temp["Saturday"] != 0) * 7 )
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 1, 'DateOfWeek'] = "Sunday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 2, 'DateOfWeek'] = "Monday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 3, 'DateOfWeek'] = "Tuesday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 4, 'DateOfWeek'] = "Wednesday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 5, 'DateOfWeek'] = "Thursday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 6, 'DateOfWeek'] = "Friday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 7, 'DateOfWeek'] = "Saturday"
for col in jan_df['EnglishDayNameOfWeek'].unique().tolist():
  day_of_week_temp.drop(col, axis=1, inplace=True)

In [48]:
jan_transaction_df = pd.merge(left=jan_transaction_df, right=day_of_week_temp, left_on='TransactionDate', right_on='Date')
jan_transaction_df.drop('Date', axis=1, inplace=True)
jan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType,DateOfWeek
0,2017-01-01,Bang Sue,186,WE,Sunday
1,2017-01-01,Chatuchak Park,359,WE,Sunday
2,2017-01-01,Hua Lamphong,299,WE,Sunday
3,2017-01-01,Huai Khwang,431,WE,Sunday
4,2017-01-01,Kamphaeng Phet,233,WE,Sunday
...,...,...,...,...,...
553,2017-01-31,Sam Yan,1241,WD,Tuesday
554,2017-01-31,Si Lom,2384,WD,Tuesday
555,2017-01-31,Sukhumvit,4891,WD,Tuesday
556,2017-01-31,Sutthisan,1968,WD,Tuesday


# Preparation ข้อมูล วันที่เท่าไหร่ของเดือน และ เป็นช่วงไหนของเดือน (ต้นเดือน , กลางเดือน , สิ้นเดือน)

In [49]:
df_day_number_of_month = pd.DataFrame(columns=['DayNumberOfMonth','MonthPeriod','MonthNumberOfYear','CalendarQuater'])
df_day_number_of_month["DayNumberOfMonth"] =  jan_transaction_df['TransactionDate'].str.slice(8, 10).astype('int')
df_day_number_of_month["MonthPeriod"] =  jan_transaction_df['TransactionDate'].str.slice(8, 10).astype('int')
df_day_number_of_month["MonthNumberOfYear"] =  jan_transaction_df['TransactionDate'].str.slice(5, 7).astype('int')

In [50]:
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'].astype('int') <= 5, 'MonthPeriod'] = 1
df_day_number_of_month.loc[( (df_day_number_of_month['MonthPeriod'].astype('int') > 5) & (df_day_number_of_month['MonthPeriod'].astype('int') < 25) ), 'MonthPeriod'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'].astype('int') >= 25, 'MonthPeriod'] = 3

df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 1, 'MonthPeriod'] = "Early"
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 2, 'MonthPeriod'] = "Mid"
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 3, 'MonthPeriod'] = "Late"

In [51]:
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 1, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 2, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 3, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 4, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 5, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 6, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 7, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 8, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 9, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 10, 'CalendarQuater'] = 4
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 11, 'CalendarQuater'] = 4
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 12, 'CalendarQuater'] = 4

In [52]:
jan_transaction_df = jan_transaction_df.join(df_day_number_of_month)
jan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType,DateOfWeek,DayNumberOfMonth,MonthPeriod,MonthNumberOfYear,CalendarQuater
0,2017-01-01,Bang Sue,186,WE,Sunday,1,Early,1,1
1,2017-01-01,Chatuchak Park,359,WE,Sunday,1,Early,1,1
2,2017-01-01,Hua Lamphong,299,WE,Sunday,1,Early,1,1
3,2017-01-01,Huai Khwang,431,WE,Sunday,1,Early,1,1
4,2017-01-01,Kamphaeng Phet,233,WE,Sunday,1,Early,1,1
...,...,...,...,...,...,...,...,...,...
553,2017-01-31,Sam Yan,1241,WD,Tuesday,31,Late,1,1
554,2017-01-31,Si Lom,2384,WD,Tuesday,31,Late,1,1
555,2017-01-31,Sukhumvit,4891,WD,Tuesday,31,Late,1,1
556,2017-01-31,Sutthisan,1968,WD,Tuesday,31,Late,1,1


# อ่านข้อมูล Events

In [53]:
events_df = pd.read_csv("gdrive/My Drive/NIDA/Events.csv")
events_df.drop('EventsId', axis=1, inplace=True)

In [54]:
events_df.head()

,EvenstName,EventsLocation,EventsType,Interest,MRTStation,EventsDate
0,LEGAL EXECUTION DEPARTMENT AUCTION 2017 Ep.1,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),กฏหมาย,2241,Queen Sirikit National Convention Centre,2017-01-08
1,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-19
2,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-20
3,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-21
4,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-22


In [55]:
events_df.MRTStation.unique()

array(['Queen Sirikit National Convention Centre', 'Phahon Yothin',
       'Phra Ram 9', 'Sukhumvit'], dtype=object)

In [56]:
jan_transaction_df.StationName.unique()

array(['Bang Sue', 'Chatuchak Park', 'Hua Lamphong', 'Huai Khwang',
       'Kamphaeng Phet', 'Khlong Toei', 'Lat Phrao', 'Lumphini',
       'Phahon Yothin', 'Phetchaburi', 'Phra Ram 9',
       'Queen Sirikit National Convention Centre', 'Ratchadaphisek',
       'Sam Yan', 'Si Lom', 'Sukhumvit', 'Sutthisan',
       'Thailand Cultural Centre'], dtype=object)

# Preparation ข้อมูล Events

In [57]:
xjan_transaction_df = pd.merge(left=jan_transaction_df,how='left', right=events_df, left_on=['StationName','TransactionDate'], right_on=['MRTStation','EventsDate'])
xjan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType,DateOfWeek,DayNumberOfMonth,MonthPeriod,MonthNumberOfYear,CalendarQuater,EvenstName,EventsLocation,EventsType,Interest,MRTStation,EventsDate
0,2017-01-01,Bang Sue,186,WE,Sunday,1,Early,1,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01,Chatuchak Park,359,WE,Sunday,1,Early,1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01,Hua Lamphong,299,WE,Sunday,1,Early,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-01,Huai Khwang,431,WE,Sunday,1,Early,1,1,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01,Kamphaeng Phet,233,WE,Sunday,1,Early,1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2017-01-31,Sam Yan,1241,WD,Tuesday,31,Late,1,1,NaN,NaN,NaN,NaN,NaN,NaN
584,2017-01-31,Si Lom,2384,WD,Tuesday,31,Late,1,1,NaN,NaN,NaN,NaN,NaN,NaN
585,2017-01-31,Sukhumvit,4891,WD,Tuesday,31,Late,1,1,NaN,NaN,NaN,NaN,NaN,NaN
586,2017-01-31,Sutthisan,1968,WD,Tuesday,31,Late,1,1,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
xjan_transaction_df.to_excel (r'Jan_Summary.xlsx', index = False, header=True)